In [12]:
import configparser
import os

In [55]:
config_dir = "./config/"
slurm_dir = "../slurm/"
batchname = "NFW_arr1_fast"
email = "hcheung5@jhu.edu"
account = "cit122"
activate_path = "/home/mcheung1/Lensing/Glworia_jax/bin/activate"
GLWORIA_PREFIX = "/home/mcheung1/Lensing/Glworia_jax/glworia_jax"

In [56]:
config_batch_dir = os.path.join(config_dir, batchname)
os.makedirs(config_batch_dir, exist_ok=True)
slurm_batch_out_dir = os.path.join(slurm_dir, 'out', batchname)
os.makedirs(slurm_batch_out_dir, exist_ok=True)

In [57]:
config = configparser.ConfigParser()
config.optionxform = str

In [58]:
lens_model_name = 'NFW'
lens_param_name = 'kappa'
lens_param_latex = r"$\kappa$"

In [59]:
config["injection_parameters"] = dict(
    mass_1=36.0,
    mass_2=29.0,
    a_1=0.4,
    a_2=0.3,
    tilt_1=0.5,
    tilt_2=1.0,
    phi_12=1.7,
    phi_jl=0.3,
    luminosity_distance=500.0,
    theta_jn=0.4,
    psi=2.659,
    phase=1.3,
    # y = 1.0,
    MLz = 500.,
    # kappa = 3.0,
    geocent_time=1126259642.413,
    ra=1.375,
    dec=-1.2108,
)

In [60]:
config['interpolator_settings'] = {
    'y_low': 0.1,
    'y_high': 5.0,
    lens_param_name + '_low': 0.1,
    lens_param_name + '_high': 10.0,
    'N_grid': 30,
    'N_grid_strong': 60,
    'N_crit': 60,
    'N': 200,
    'lens_model_name': lens_model_name,

    'T0_max': 1000,

    'y_low_im': 0.1,
    'y_high_im': 5.0,
    lens_param_name + '_low_im': 0.1,
    lens_param_name + '_high_im': 10.0,
    'N_grid_im': 1000,
    'N_crit_im': 1000,
}

In [61]:
config['prior_settings'] = {
    'MLz_prior_type': 'uniform',
    'MLz_min': 1.0,
    'MLz_max': 2000.0,
    'y_prior_type': 'uniform',
    'y_min': 0.1,
    'y_max': 5.0,
    lens_param_name + '_prior_type': 'uniform',
    lens_param_name + '_min': 0.1,
    lens_param_name + '_max': 10.0,
}

In [62]:
config['waveform_arguments'] = dict(
    waveform_approximant="IMRPhenomPv2",
    reference_frequency=50.0,
    minimum_frequency=20.0)

In [63]:
config['misc'] = { 
    'duration': 4.0,
    'sampling_frequency': 2048.0,
    'minimum_frequency': 20,
    'seed': 88170235,
    # 'outdir_ext': '',
    'lp_name': lens_param_name,
    'lp_latex': lens_param_latex,
}

In [64]:
config['sampler_settings'] = {
    'sampler': 'dynesty',
    'nlive': 1000}

In [65]:
y_arr = [1.0, 2.0, 3.0]
kappa_arr = [0.5, 1.0, 5.0]

config_file_path_list = []

for y in y_arr:
    config["injection_parameters"]["y"] = f'{y:.4f}'
    for kappa in kappa_arr:
        config["injection_parameters"]["kappa"] = f'{kappa:.4f}'
        config_file_path = os.path.join(config_batch_dir, f"MLz_500_y_{y:.4f}_kappa_{kappa:.4f}.ini")

        config['misc']['outdir_ext'] = batchname
        with open(config_file_path, 'w') as configfile:
            config.write(configfile)
        config_file_path_list.append(os.path.abspath(config_file_path))

In [66]:
config_file_path_list

['/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_1.0000_kappa_0.5000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_1.0000_kappa_1.0000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_1.0000_kappa_5.0000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_2.0000_kappa_0.5000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_2.0000_kappa_1.0000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_2.0000_kappa_5.0000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_3.0000_kappa_0.5000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glworia_jax/bilby_scripts/config/NFW_arr1_fast/MLz_500_y_3.0000_kappa_1.0000.ini',
 '/home/mcheung1/Lensing/Glworia_jax/glw

In [67]:
config_file_path_list_file = os.path.join(config_batch_dir, "file_path.txt")
with open(config_file_path_list_file, 'w') as f:
    for item in config_file_path_list:
        f.write("%s\n" % item)

In [68]:
cpus_per_task = 8

In [69]:
slurm_script = f"""#!/bin/bash
#SBATCH --job-name="{batchname}"
#SBATCH --output="{os.path.abspath(slurm_batch_out_dir)}/%A.%a.out"
#SBATCH --partition=shared
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task={cpus_per_task}
#SBATCH --mem={cpus_per_task*2}Gb
#SBATCH --account={account}
#SBATCH -t 48:00:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user={email}
#SBATCH --array=0-{len(config_file_path_list)-1}

pwd; hostname; date

source {activate_path}
export PYTHONPATH=$PYTHONPATH:{GLWORIA_PREFIX}
export GLWORIA_PREFIX={GLWORIA_PREFIX}

mapfile -t config_file_path_list < {os.path.abspath(config_file_path_list_file)}
config_file_path=${{config_file_path_list[$SLURM_ARRAY_TASK_ID]}}
echo $config_file_path

python {GLWORIA_PREFIX}/bilby_scripts/PE_fast.py $config_file_path"""

In [70]:
slurm_script_path = os.path.join(slurm_dir, f"{batchname}.sh")
with open(slurm_script_path, 'w') as f:
    f.write(slurm_script)